In [1]:
"""Evaluation metric for Santa 2024."""

import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch
import torch.nn.functional as F


os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ParticipantVisibleError(Exception):
    pass


def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
    load_in_8bit: bool = False,
    clear_mem: bool = False,
) -> float:
    """
    Calculates the mean perplexity of submitted text permutations compared to an original text.

    Parameters
    ----------
    solution : DataFrame
        DataFrame containing the original text in a column named 'text'.
        Includes a row ID column specified by `row_id_column_name`.

    submission : DataFrame
        DataFrame containing the permuted text in a column named 'text'.
        Must have the same row IDs as the solution.
        Includes a row ID column specified by `row_id_column_name`.

    row_id_column_name : str
        Name of the column containing row IDs.
        Ensures aligned comparison between solution and submission.

    model_path : str, default='/kaggle/input/gemma-2/transformers/gemma-2-9b/2'
        Path to the serialized LLM.

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    clear_mem : bool, default=False
        Clear GPU memory after scoring by clearing the CUDA cache.
        Useful for testing.

    Returns
    -------
    float
        The mean perplexity score. Lower is better.

    Raises
    ------
    ParticipantVisibleError
        If the submission format is invalid or submitted strings are not valid permutations.

    Examples
    --------
    >>> import pandas as pd
    >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
    >>> solution = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["this is a normal english sentence", "the quick brown fox jumps over the lazy dog"]
    ... })
    >>> submission = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["sentence english normal a is this", "lazy the over jumps fox brown quick the dog"]
    ... })
    >>> score(solution, submission, 'id', model_path=model_path, clear_mem=True) > 0
    True
    """
    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)
    invalid_mask = sol_counts != sub_counts
    if invalid_mask.any():
        raise ParticipantVisibleError(
            'At least one submitted string is not a valid permutation of the solution string.'
        )

    # Calculate perplexity for the submitted strings
    sub_strings = [
        ' '.join(s.split()) for s in submission['text'].tolist()
    ]  # Split and rejoin to normalize whitespace
    scorer = PerplexityCalculator(
        model_path=model_path,
        load_in_8bit=load_in_8bit,
    )  # Initialize the perplexity calculator with a pre-trained model
    perplexities = scorer.get_perplexity(
        sub_strings
    )  # Calculate perplexity for each submitted string

    if clear_mem:
        # Just move on if it fails. Not essential if we have the score.
        try:
            scorer.clear_gpu_memory()
        except:
            print('GPU memory clearing failed.')

    return float(np.mean(perplexities))


class PerplexityCalculator:
    """
    Calculates perplexity of text using a pre-trained language model.

    Adapted from https://github.com/asahi417/lmppl/blob/main/lmppl/ppl_recurrent_lm.py

    Parameters
    ----------
    model_path : str
        Path to the pre-trained language model

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    device_map : str, default="auto"
        Device mapping for the model.
    """

    def __init__(
        self,
        model_path: str,
        load_in_8bit: bool = False,
        device_map: str = 'auto',
    ):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
        # Configure model loading based on quantization setting and device availability
        if load_in_8bit:
            if DEVICE.type != 'cuda':
                raise ValueError('8-bit quantization requires CUDA device')
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                device_map=device_map,
            )
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if DEVICE.type == 'cuda' else torch.float32,
                device_map=device_map,
            )

        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')

        self.model.eval()

    def get_perplexity(
        self, input_texts: Union[str, List[str]], debug=False
    ) -> Union[float, List[float]]:
        """
        Calculates the perplexity of given texts.

        Parameters
        ----------
        input_texts : str or list of str
            A single string or a list of strings.

        batch_size : int, default=None
            Batch size for processing. Defaults to the number of input texts.

        debug : bool, default=False
            Print debugging information.

        Returns
        -------
        float or list of float
            A single perplexity value if input is a single string,
            or a list of perplexity values if input is a list of strings.

        Examples
        --------
        >>> import pandas as pd
        >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
        >>> scorer = PerplexityCalculator(model_path=model_path)

        >>> submission = pd.DataFrame({
        ...     'id': [0, 1, 2],
        ...     'text': ["this is a normal english sentence", "thsi is a slihgtly misspelled zr4g sentense", "the quick brown fox jumps over the lazy dog"]
        ... })
        >>> perplexities = scorer.get_perplexity(submission["text"].tolist())
        >>> perplexities[0] < perplexities[1]
        True
        >>> perplexities[2] < perplexities[0]
        True

        >>> perplexities = scorer.get_perplexity(["this is a sentence", "another sentence"])
        >>> all(p > 0 for p in perplexities)
        True

        >>> scorer.clear_gpu_memory()
        """
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []
        with torch.no_grad():
            # Process each sequence independently
            for text in input_texts:
                # Explicitly add sequence boundary tokens to the text
                text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"

                # Tokenize
                model_inputs = self.tokenizer(
                    text_with_special,
                    return_tensors='pt',
                    add_special_tokens=False,
                )

                if 'token_type_ids' in model_inputs:
                    model_inputs.pop('token_type_ids')

                model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                # Get model output
                output = self.model(**model_inputs, use_cache=False)
                logits = output['logits']

                # Shift logits and labels for calculating loss
                shift_logits = logits[..., :-1, :].contiguous()  # Drop last prediction
                shift_labels = model_inputs['input_ids'][..., 1:].contiguous()  # Drop first input

                # Calculate token-wise loss
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                # Calculate average loss
                sequence_loss = loss.sum() / len(loss)
                loss_list.append(sequence_loss.cpu().item())

                # Debug output
                if debug:
                    print(f"\nProcessing: '{text}'")
                    print(f"With special tokens: '{text_with_special}'")
                    print(f"Input tokens: {model_inputs['input_ids'][0].tolist()}")
                    print(f"Target tokens: {shift_labels[0].tolist()}")
                    print(f"Input decoded: {self.tokenizer.decode(model_inputs['input_ids'][0])}")
                    print(f"Target decoded: {self.tokenizer.decode(shift_labels[0])}")
                    print(f"Individual losses: {loss.tolist()}")
                    print(f"Average loss: {sequence_loss.item():.4f}")

        ppl = [exp(i) for i in loss_list]

        if debug:
            print("\nFinal perplexities:")
            for text, perp in zip(input_texts, ppl):
                print(f"Text: '{text}'")
                print(f"Perplexity: {perp:.2f}")

        return ppl[0] if single_input else ppl

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        # Delete model and tokenizer if they exist
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'tokenizer'):
            del self.tokenizer

        # Run garbage collection
        gc.collect()

        # Clear CUDA cache and reset memory stats
        with DEVICE:
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()
    
    def get_word_transition_matrix(self, sentence: str, debug=False):
        """
        Calculate the transition probability matrix for words in a sentence.
    
        Parameters
        ----------
        sentence : str
            The input sentence for which the transition probabilities are calculated.
    
        debug : bool, default=False
            Print debugging information.
    
        Returns
        -------
        pd.DataFrame
            A DataFrame containing the transition probabilities between words.
        """
        # Add special tokens to the input sentence
        text_with_special = f"{self.tokenizer.bos_token}{sentence}{self.tokenizer.eos_token}"
    
        if debug:
            print(f"Input sentence: {sentence}")
            print(f"Sentence with special tokens: {text_with_special}")
    
        # Tokenize the sentence
        model_inputs = self.tokenizer(
            text_with_special,
            return_tensors='pt',
            add_special_tokens=False,
        )
    
        if 'token_type_ids' in model_inputs:
            model_inputs.pop('token_type_ids')
    
        model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}
    
        if debug:
            print(f"Tokenized input IDs: {model_inputs['input_ids'][0].tolist()}")
            print(f"Decoded tokens: {self.tokenizer.convert_ids_to_tokens(model_inputs['input_ids'][0].tolist())}")
    
        with torch.no_grad():
            # Get model output
            output = self.model(**model_inputs, use_cache=False)
            logits = output['logits']  # Shape: (batch_size, seq_len, vocab_size)
    
        if debug:
            print(f"Logits shape: {logits.shape}")
    
        # Convert logits to probabilities using softmax
        probabilities = F.softmax(logits, dim=-1).squeeze(0)  # Shape: (seq_len, vocab_size)
    
        if debug:
            print(f"Softmax probabilities shape: {probabilities.shape}")
    
        # Map token IDs to words
        tokens = model_inputs['input_ids'][0].tolist()
        words = self.tokenizer.convert_ids_to_tokens(tokens)
    
        if debug:
            print(f"Tokens: {tokens}")
            print(f"Words: {words}")
    
        # Build transition probability matrix
        transition_matrix = []
        for i, word in enumerate(words[:-1]):  # Skip the last token as it has no next word
            next_probs = probabilities[i].cpu().numpy()  # Probabilities for the next token
            transition_matrix.append((word, next_probs))
    
            if debug:
                print(f"Word: {word}")
                print(f"Next token probabilities: {next_probs}")
    
        # Convert to DataFrame for easier readability
        vocab_words = self.tokenizer.convert_ids_to_tokens(range(len(next_probs)))
        df = pd.DataFrame(
            {word: probs for word, probs in transition_matrix},
            index=vocab_words
        ).T
    
        if debug:
            print(f"Transition matrix DataFrame:\n{df}")
    
        return df.loc[words[1:-1], words[1:-1]]



In [2]:
scorer = PerplexityCalculator(model_path="/kaggle/input/gemma-2/transformers/gemma-2-9b/2")

sentence = "the quick brown fox jumps over the lazy dog"
transition_matrix = scorer.get_word_transition_matrix(sentence, True)

print(transition_matrix)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Input sentence: the quick brown fox jumps over the lazy dog
Sentence with special tokens: <bos>the quick brown fox jumps over the lazy dog<eos>
Tokenized input IDs: [2, 1175, 4320, 8426, 25341, 36271, 1163, 573, 27894, 5929, 1]
Decoded tokens: ['<bos>', 'the', '▁quick', '▁brown', '▁fox', '▁jumps', '▁over', '▁the', '▁lazy', '▁dog', '<eos>']
Logits shape: torch.Size([1, 11, 256000])
Softmax probabilities shape: torch.Size([11, 256000])
Tokens: [2, 1175, 4320, 8426, 25341, 36271, 1163, 573, 27894, 5929, 1]
Words: ['<bos>', 'the', '▁quick', '▁brown', '▁fox', '▁jumps', '▁over', '▁the', '▁lazy', '▁dog', '<eos>']
Word: <bos>
Next token probabilities: [8.7685074e-11 2.8206819e-06 2.3340651e-06 ... 9.1015817e-10 5.7863359e-09
 8.7873937e-11]
Word: the
Next token probabilities: [1.5185595e-21 1.2502646e-07 8.4630794e-17 ... 8.9107701e-18 4.6668081e-17
 2.5440984e-21]
Word: ▁quick
Next token probabilities: [2.2997401e-22 5.6259658e-08 6.5498395e-14 ... 4.6967011e-17 8.7147481e-17
 1.8868373e-20]


In [3]:
transition_matrix

,the,▁quick,▁brown,▁fox,▁jumps,▁over,▁the,▁lazy,▁dog
the,1.783058e-07,0.000027,0.000003,0.000004,4.135287e-10,0.000005,0.001885,7.967157e-07,0.000055
▁quick,3.573963e-07,0.000112,0.253200,0.000200,5.550260e-06,0.000017,0.000453,1.292991e-06,0.000003
▁brown,1.002134e-08,0.000010,0.000014,0.940248,6.980826e-05,0.000016,0.000137,3.494785e-05,0.015788
▁fox,1.855237e-07,0.000029,0.000088,0.000693,6.046017e-01,0.001072,0.000990,9.191806e-05,0.000037
▁jumps,3.800448e-07,0.000030,0.000019,0.000009,1.235868e-05,0.969625,0.001350,1.667308e-05,0.000024
▁over,4.139934e-04,0.000023,0.000014,0.000004,2.068456e-06,0.000040,0.926231,1.333397e-02,0.000111
▁the,3.671033e-07,0.001038,0.000192,0.000027,4.187962e-06,0.000020,0.000391,9.630541e-01,0.005644
▁lazy,2.491684e-07,0.000006,0.000913,0.000361,4.308223e-06,0.000002,0.000104,1.450001e-04,0.953291
▁dog,1.944718e-04,0.000523,0.000056,0.000004,1.766617e-04,0.000025,0.009295,4.737631e-05,0.001805


---

In [4]:
model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
load_in_8bit = False
device_map: str = 'auto'
sentence = "the quick brown fox jumps over the lazy dog"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

if load_in_8bit:
    if DEVICE.type != 'cuda':
        raise ValueError('8-bit quantization requires CUDA device')
    quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=quantization_config,
        device_map=device_map,
    )
else:
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 if DEVICE.type == 'cuda' else torch.float32,
        device_map=device_map,
    )

loss_fct = torch.nn.CrossEntropyLoss(reduction='none')

model.eval()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((3584,), 

In [5]:
text_with_special = f"{tokenizer.bos_token}{sentence}{tokenizer.eos_token}"
    
print(f"Input sentence: {sentence}")
print(f"Sentence with special tokens: {text_with_special}")

Input sentence: the quick brown fox jumps over the lazy dog
Sentence with special tokens: <bos>the quick brown fox jumps over the lazy dog<eos>


In [6]:

# Tokenize the sentence
model_inputs = tokenizer(
    text_with_special,
    return_tensors='pt',
    add_special_tokens=False,
)

model_inputs

{'input_ids': tensor([[    2,  1175,  4320,  8426, 25341, 36271,  1163,   573, 27894,  5929,
             1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
if 'token_type_ids' in model_inputs:
    model_inputs.pop('token_type_ids')


In [8]:

model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

print(f"Tokenized input IDs: {model_inputs['input_ids'][0].tolist()}")
print(f"Decoded tokens: {tokenizer.convert_ids_to_tokens(model_inputs['input_ids'][0].tolist())}")


Tokenized input IDs: [2, 1175, 4320, 8426, 25341, 36271, 1163, 573, 27894, 5929, 1]
Decoded tokens: ['<bos>', 'the', '▁quick', '▁brown', '▁fox', '▁jumps', '▁over', '▁the', '▁lazy', '▁dog', '<eos>']


In [9]:
with torch.no_grad():
    # Get model output
    output = model(**model_inputs, use_cache=False)
    logits = output['logits']  # Shape: (batch_size, seq_len, vocab_size)

print(f"Logits shape: {logits.shape}")

Logits shape: torch.Size([1, 11, 256000])


In [10]:
# Convert logits to probabilities using softmax
probabilities = F.softmax(logits, dim=-1).squeeze(0)  # Shape: (seq_len, vocab_size)

print(f"Softmax probabilities shape: {probabilities.shape}")

Softmax probabilities shape: torch.Size([11, 256000])


In [11]:
probabilities[0][1]

tensor(2.8207e-06)

In [12]:
# Map token IDs to words
tokens = model_inputs['input_ids'][0].tolist()
words = tokenizer.convert_ids_to_tokens(tokens)

print(f"Tokens: {tokens}")
print(f"Words: {words}")

Tokens: [2, 1175, 4320, 8426, 25341, 36271, 1163, 573, 27894, 5929, 1]
Words: ['<bos>', 'the', '▁quick', '▁brown', '▁fox', '▁jumps', '▁over', '▁the', '▁lazy', '▁dog', '<eos>']


In [13]:
# Build transition probability matrix
transition_matrix = []
for i, word in enumerate(words[:-1]):  # Skip the last token as it has no next word
    next_probs = probabilities[i].cpu().numpy()  # Probabilities for the next token
    transition_matrix.append((word, next_probs))

print(f"Word: {word}")
print(f"Next token probabilities: {next_probs}")

Word: ▁dog
Next token probabilities: [7.7320305e-19 5.8210123e-04 4.9169313e-09 ... 9.3961212e-15 1.4295166e-10
 1.6468671e-18]


In [14]:
transition_matrix[0][1]

array([8.7685074e-11, 2.8206819e-06, 2.3340651e-06, ..., 9.1015817e-10,
       5.7863359e-09, 8.7873937e-11], dtype=float32)

In [15]:
# Convert to DataFrame for easier readability
vocab_words = tokenizer.convert_ids_to_tokens(range(len(next_probs)))

In [16]:
df = pd.DataFrame(
    {word: probs for word, probs in transition_matrix},
    index=vocab_words
).T

df

,<pad>,<eos>,<bos>,<unk>,<mask>,<2mass>,[@BOS@],<unused0>,<unused1>,<unused2>,...,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t,<unused99>
<bos>,8.768507e-11,2.820682e-06,2.334065e-06,9.729913e-11,9.833538e-10,8.803667e-11,8.781731e-11,8.965530e-11,8.776539e-11,9.336165e-11,...,2.981904e-09,2.359339e-09,1.932901e-09,2.028075e-09,1.207352e-09,1.413002e-09,1.241720e-09,9.101582e-10,5.786336e-09,8.787394e-11
the,1.518560e-21,1.250265e-07,8.463079e-17,3.530991e-18,3.574210e-17,2.325175e-21,1.684262e-21,1.102790e-20,1.255812e-21,1.320553e-18,...,4.025468e-16,2.794661e-18,6.072353e-18,4.905005e-19,1.429575e-18,2.817882e-17,3.948116e-18,8.910770e-18,4.666808e-17,2.544098e-21
▁quick,2.299740e-22,5.625966e-08,6.549839e-14,8.971059e-19,6.743648e-16,1.874583e-21,4.002011e-22,1.517974e-20,2.724661e-22,6.146432e-19,...,9.627933e-16,5.932774e-16,2.795282e-15,6.538876e-15,1.257670e-16,3.175455e-14,2.650639e-17,4.696701e-17,8.714748e-17,1.886837e-20
▁brown,2.747267e-18,9.279092e-07,9.973323e-09,8.810970e-16,5.135275e-13,1.356851e-17,3.286267e-18,6.326745e-17,3.175357e-18,4.891133e-16,...,1.052934e-13,4.278197e-13,3.756161e-12,3.270806e-12,4.011927e-12,1.234083e-12,2.522628e-12,1.146441e-12,2.326677e-12,3.606576e-17
▁fox,4.471759e-18,9.848375e-06,7.439464e-09,1.364290e-15,5.811374e-11,6.380579e-17,4.991959e-18,7.068722e-17,5.304070e-18,2.402501e-15,...,7.250182e-13,9.973447e-12,1.332511e-11,1.683319e-11,2.827918e-12,3.214613e-12,2.476189e-12,3.069991e-13,6.192220e-11,1.965619e-16
▁jumps,4.613748e-15,1.451139e-06,2.968402e-07,2.681263e-14,6.713349e-11,4.891192e-15,5.173109e-15,1.124034e-14,4.704208e-15,1.953201e-14,...,3.911236e-10,8.460178e-10,5.343395e-10,4.211394e-10,2.618327e-10,1.752990e-10,3.425694e-10,3.853581e-10,1.522674e-09,3.651962e-15
▁over,3.599281e-13,3.490728e-06,1.040482e-07,8.185314e-13,3.933180e-10,3.575858e-13,3.829770e-13,5.106124e-13,3.629393e-13,6.556368e-13,...,9.829776e-10,1.273034e-09,1.144414e-09,1.577922e-09,1.952537e-09,5.938217e-10,5.492665e-10,5.182685e-10,1.453623e-09,3.056243e-13
▁the,1.592185e-18,1.281721e-06,1.371278e-09,7.404272e-16,1.856211e-12,1.822784e-17,1.849617e-18,1.443125e-16,1.835896e-18,4.620167e-16,...,6.149456e-12,1.612323e-11,9.873565e-12,2.845834e-11,7.265125e-11,2.623685e-12,4.568319e-12,6.765224e-13,1.048265e-11,5.397812e-17
▁lazy,3.111976e-17,1.200957e-06,3.271050e-08,5.999824e-15,3.416491e-11,7.264350e-17,4.005685e-17,5.957673e-16,3.217099e-17,2.752311e-15,...,1.723762e-11,3.302691e-11,8.057585e-11,1.636022e-10,9.007577e-11,2.410997e-11,6.882323e-11,4.258060e-11,6.204002e-11,5.128041e-17
▁dog,7.732030e-19,5.821012e-04,4.916931e-09,4.367617e-16,8.965697e-13,3.386236e-18,1.353219e-18,2.286275e-17,8.210760e-19,5.884405e-16,...,1.249314e-12,9.855822e-12,5.362627e-12,7.140691e-13,1.172661e-13,3.094458e-13,5.057560e-14,9.396121e-15,1.429517e-10,1.646867e-18


In [17]:
words[1:-1]

['the', '▁quick', '▁brown', '▁fox', '▁jumps', '▁over', '▁the', '▁lazy', '▁dog']

In [18]:
df.loc[words[1:-1], words[1:-1]]


,the,▁quick,▁brown,▁fox,▁jumps,▁over,▁the,▁lazy,▁dog
the,1.783058e-07,0.000027,0.000003,0.000004,4.135287e-10,0.000005,0.001885,7.967157e-07,0.000055
▁quick,3.573963e-07,0.000112,0.253200,0.000200,5.550260e-06,0.000017,0.000453,1.292991e-06,0.000003
▁brown,1.002134e-08,0.000010,0.000014,0.940248,6.980826e-05,0.000016,0.000137,3.494785e-05,0.015788
▁fox,1.855237e-07,0.000029,0.000088,0.000693,6.046017e-01,0.001072,0.000990,9.191806e-05,0.000037
▁jumps,3.800448e-07,0.000030,0.000019,0.000009,1.235868e-05,0.969625,0.001350,1.667308e-05,0.000024
▁over,4.139934e-04,0.000023,0.000014,0.000004,2.068456e-06,0.000040,0.926231,1.333397e-02,0.000111
▁the,3.671033e-07,0.001038,0.000192,0.000027,4.187962e-06,0.000020,0.000391,9.630541e-01,0.005644
▁lazy,2.491684e-07,0.000006,0.000913,0.000361,4.308223e-06,0.000002,0.000104,1.450001e-04,0.953291
▁dog,1.944718e-04,0.000523,0.000056,0.000004,1.766617e-04,0.000025,0.009295,4.737631e-05,0.001805
